In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import re
import pstats as ps

In [ ]:
MODELS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models")
PLOTS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/plots")
ANALYSIS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/analysis")
exclude=["test_run_00013", "test_run_00014"]

In [35]:
def read_profile(path: Path, rank: bool = False):
    """
    Function to parse a 'FIT Profiler Report' from a textfile. 

    Returns:
    tuple: (
        df: pd.DataFrame containing all profiling data,
        df_sum: pd.DataFrame containing data for the function)
    """
    print(f"Parsing file: {path}")
    with open(path, "r") as rf:
        parsed_file = rf.read()
    parsed_file = parsed_file.split("\n\n\n")
    
    column_names = ["ncalls", "tot_time", "tot_percall", "cum_time", "cum_percall", "filename:lineno(function)", "Class", "Function", "Call_num", "Primitive_Call_num", "Time", "Rank", "Summary_index",]
    
    class_pattern = re.compile("\[\S+(?=\.)")
    function_pattern = re.compile("\w+(?=#)")
    callnum_pattern = re.compile("(?<=\s\s\s)\d+")
    primcallnum_pattern = re.compile("(?<=\()\d+")
    time_pattern= re.compile("\d+\.\d\d\d")
    rank_pattern = re.compile(":\s\d*\s")
    patterns = [class_pattern, function_pattern, callnum_pattern, primcallnum_pattern, time_pattern, rank_pattern]

    df_sum = pd.DataFrame(columns=column_names[6:-1])
    df = pd.DataFrame(columns=column_names)
    for n, section in enumerate(parsed_file):
        section = section.split("\n")
        title = "#".join(section[1:3])

        title_columns = [re.findall(pat, title) for pat in patterns]
        title_columns = [str(s[0]) if len(s)>0 else pd.NA for s in title_columns]
        df_sum.loc[len(df_sum)] = title_columns
        
        title_columns.append(len(df_sum)-1)
        for line in section[7:]:
            
            split = line.split(" ")
            while "" in split:
                split.remove("")
            if "/" in split[0]:
                split[0] = split[0].split("/")[0]

            df.loc[len(df), column_names[:5]] = split[:5]
            if re.findall("{.+}", line) != []:
                #print(re.findall("{.+}", line))
                df.iloc[len(df)-1, 5] = str(re.findall("{.+}", line)[0])
            else:
                #print("split", split[-1])
                df.iloc[len(df)-1, 5] = split[-1]

            df.iloc[len(df)-1, 6:] = title_columns
        
            
    return df, df_sum.iloc[:-1]

#read_profile("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00030/profiler/fit-training_profile.txt")[1].iloc[-3:]

In [ ]:
def filter_profiler_records(df, df_sum):
    print("Filtering...")
    df_sum=df_sum.astype({"Time": float,
                          "Call_num": int})
    df = df.astype({"tot_time": float,
                        "ncalls": int})
    clean_df = pd.DataFrame(columns=df.columns)
    for func_index, function in df_sum.Function.items():
        profiler_time = 0.0
        profiler_calls = 0
        sub_sel = df.loc[df["Function"] == function]
        for _, row in sub_sel.iterrows():
            if "profiler".casefold() in row["filename:lineno(function)"].casefold():
                profiler_time += row["tot_time"]
                profiler_calls += row["ncalls"]
                continue
            else:
                clean_df.loc[len(clean_df)] = row
        df_sum.loc[func_index, "Time"] -= profiler_time
        df_sum.loc[func_index, "Call_num"] -= profiler_calls
    return df_sum,clean_df

In [34]:
# parse the models dict for profiler data from testrun_synthetic_benchmark.py
profile_dict = dict()
hyperparameter_names= ["run_id","data_n_genes", "data_n_samples_control", "data_n_samples_per_perturbation", "batch_size", "n_epochs", "n_epochs_pretrain_latents", "scale_factor"]
hyperparameters = pd.DataFrame(columns=hyperparameter_names)


for subdir in MODELS_PATH.iterdir():
    if subdir.joinpath("profiler", "fit-training_profile.txt").exists():
        key = str(subdir.name)
        if key in exclude:
            print(f"Skipping {key}!")
            continue
        print(subdir)
        # get globals of run
        globs = pd.read_csv(PLOTS_PATH.joinpath(key, "globals.csv"), delimiter=",").set_index("0", drop=True).T
        available_paras = [n for n in hyperparameter_names if n in globs.columns]
        hyperparameters.loc[len(hyperparameters)] = globs[available_paras].iloc[1]

        df, df_sum = read_profile(subdir.joinpath("profiler", "fit-training_profile.txt"))

        # remove profiler traces
        df_sum, clean_df = filter_profiler_records(df, df_sum)

        profile_dict[key] = dict()
        profile_dict[key]["full_training_profile"] = clean_df
        profile_dict[key]["training_profile"] = df_sum
        if subdir.joinpath("profiler/fit-pretraining_profile.txt").exists():
            df, df_sum = read_profile(subdir.joinpath("profiler/fit-pretraining_profile.txt"))


            df_sum, clean_df = filter_profiler_records(df, df_sum)

            profile_dict[key]["full_pretraining_profile"] = df
            profile_dict[key]["pretraining_profile"] = df_sum


/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00032
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00032/profiler/fit-training_profile.txt
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00032/profiler/fit-pretraining_profile.txt
/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00030
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00030/profiler/fit-training_profile.txt
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00030/profiler/fit-pretraining_profile.txt
/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00031
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run_00031/profiler/fit-training_profile.txt
Parsing file: /data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models/test_run

In [ ]:
def process_profile_data(df, df_sum):
    full_dtype_converter = {
    "ncalls": int,
    "tot_time": float,
    "tot_percall":float,
    "cum_time":float,
    "cum_percall":float,
    "filename:lineno(function)":str,
    "Class":str,
    "Function":str,
    "Call_num":int,
    "Primitive_Call_num":int,
    "Time":float,
    #"Rank":int,
    "Summary_index": int
}
    dtype_converter = {
    "Class":str,
    "Function":str,
    "Call_num":int,
    "Primitive_Call_num":int,
    "Time":float,
    #"Rank":int,
}
    df_sum["Primitive_Call_num"] = df_sum["Primitive_Call_num"].fillna(0)
    df_sum = df_sum.astype(dtype_converter)
    df_sum["Class_Function"] = df_sum["Class"] + "_" + df_sum["Function"]
    df_sum["is_callback"] = df_sum["Class"].apply(lambda x: "Callback" in x)

    df["Primitive_Call_num"] = df["Primitive_Call_num"].fillna(0)
    df = df.astype(full_dtype_converter)
    df["Class_Function"] = df["Class"] + "_" + df["Function"]
    df["filename_lineno(function)"] = df["filename:lineno(function)"]
    df.drop(columns=["filename:lineno(function)"])
    df["Class_Function_etc"] = df["Class_Function"] + ":" + df["filename_lineno(function)"]
    df["is_callback"] = df["Class"].apply(lambda x: "Callback" in x)

    return df, df_sum


In [ ]:
# parse the models dict for profiler data from testrun_synthetic_benchmark.py
def fullparse_profiles(
        pickle:bool=False,
        MODELS_PATH: Path = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models"),
        ANALYSIS_PATH:Path=Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/analysis"),
        hyperparameter_names: list = ["run_id","data_n_genes", "data_n_samples_control", "data_n_samples_per_perturbation", "batch_size", "n_epochs", "n_epochs_pretrain_latents", "scale_factor"],
    ):
    """
    Standard run to parse all profile output files in MODELS_PATH inside subdirectories with name "test_run_<run_id>".

    Returns:
    tuple(
        hyperparameters: pd.DataFrame containing run hyperparameters for each id,
        profile_dict: dict() containing DataFrames for all training and pretraining runs
    )
    """

    profile_dict = dict()
    hyperparameters = pd.DataFrame(columns=hyperparameter_names)


    for subdir in MODELS_PATH.iterdir():
        if subdir.joinpath("profiler", "fit-training_profile.txt").exists():
            key = str(subdir.name)
            if key in exclude:
                print(f"Skipping {key}!")
                continue
            print(subdir)
        # get globals of run
            globs = pd.read_csv(PLOTS_PATH.joinpath(key, "globals.csv"), delimiter=",").set_index("0", drop=True).T
            available_paras = [n for n in hyperparameter_names if n in globs.columns]
            hyperparameters.loc[len(hyperparameters)] = globs[available_paras].iloc[1]

            df, df_sum = read_profile(subdir.joinpath("profiler", "fit-training_profile.txt"))

        # remove profiler traces
            df_sum, clean_df = filter_profiler_records(df, df_sum)


            df, df_sum = process_profile_data(df, df_sum)
            if pickle:
                clean_df.to_pickle(ANALYSIS_PATH.joinpath(key, "full_training_profile.gz"))
                df_sum.to_pickle(ANALYSIS_PATH.joinpath(key, "training_profile.gz"))
            profile_dict[key] = dict()
            profile_dict[key]["full_training_profile"] = clean_df
            profile_dict[key]["training_profile"] = df_sum

            # check for pretraining profiles
            if subdir.joinpath("profiler/fit-pretraining_profile.txt").exists():
                df, df_sum = read_profile(subdir.joinpath("profiler/fit-pretraining_profile.txt"))


                df_sum, clean_df = filter_profiler_records(df, df_sum)

                df, df_sum = process_profile_data(df, df_sum)

                if pickle:
                    df.to_pickle(ANALYSIS_PATH.joinpath(key, "full_pretraining_profile.gz"))
                    df_sum.to_pickle(ANALYSIS_PATH.joinpath(key, "pretraining_profile.gz"))

                profile_dict[key]["full_pretraining_profile"] = df
                profile_dict[key]["pretraining_profile"] = df_sum
    if pickle:
        hyperparameters.to_csv(ANALYSIS_PATH.joinpath("parameters.csv"))

    return hyperparameters, profile_dict


In [36]:
full_dtype_converter = {
    "ncalls": int,
    "tot_time": float,
    "tot_percall":float,
    "cum_time":float,
    "cum_percall":float,
    "filename:lineno(function)":str,
    "Class":str,
    "Function":str,
    "Call_num":int,
    "Primitive_Call_num":int,
    "Time":float,
    #"Rank":int,
    "Summary_index": int
}
dtype_converter = {
    "Class":str,
    "Function":str,
    "Call_num":int,
    "Primitive_Call_num":int,
    "Time":float,
    #"Rank":int,
}
# type conversion and saving
for n, d in profile_dict.items():
    for name, df in d.items():
        df["Primitive_Call_num"] = df["Primitive_Call_num"].fillna(0)
        if name.startswith("full"):
            df = df.astype(full_dtype_converter)
        else:
            df = df.astype(dtype_converter)
        df["Class_Function"] = df["Class"] + "_" + df["Function"]
        if name.startswith("full"):
            df["filename_lineno(function)"] = df["filename:lineno(function)"]
            df.drop(columns=["filename:lineno(function)"])
            df["Class_Function_etc"] = df["Class_Function"] + ":" + df["filename_lineno(function)"]
        df["is_callback"] = df["Class"].apply(lambda x: "Callback" in x)

        ANALYSIS_PATH.joinpath(n).mkdir(parents=True, exist_ok=True)
        df.to_pickle(ANALYSIS_PATH.joinpath(n,f"{name}.gz"))
        profile_dict[n][name] = df
hyperparameters.to_csv(ANALYSIS_PATH.joinpath("parameters.csv"))